# Value-at-Risk for Currencies

### Lecture Notes by Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MATH 100.2: Topics in Financial Mathematics II \
First Semester, S.Y. 2021-2022 \
Ateneo de Manila University

In [1]:
import numpy as np # Numerical Computing
import pandas as pd # Data wrangling
import matplotlib.pyplot as plt # Plotting
from statistics import NormalDist # statistical analysis

%matplotlib inline

## Helper functions from previous programming sessions

In [2]:
def get_return(df, d):
    """
    df is the original df
    
    appends returns series to df
    """
    df["previous"] = df["close"].shift(-d)
    df["return"] = np.log(df["close"]/df["previous"])
    return df 

In [3]:
def get_ewma_weights_df(df, d, ewma_par):
    """ 
    appends weights series to df
    """
    count_returns = len(df["close"])-d # gives the number of non-NaN (or non-empty) returns
    weight_lst = [(1-ewma_par)*(ewma_par**j) for j in range(count_returns)]
    df["weight"] = pd.Series(weight_lst)
    return df

In [4]:
def get_change_in_value_df(df, N):
    """
    df is the output of get_returns_df 
    
    appends change in portfolio value series to df
    """
    S0 = df.loc[0, "close"]
    df["change_in_value"] = N * S0 * df["return"]
    return df

In [5]:
def get_kth_percentile_discrete(df, d, alpha):
    """
    this returns the (1-alpha)th percentile of the ordered array of historical changes in portfolio values
    """
    M = len(df)-d
    k = int(np.floor((1-alpha)*M))
    var = abs(df["change_in_value"].sort_values(ignore_index=True)[k-1])
    
    return var

In [6]:
def get_brw_weights_df(df, d, decay_par):
    """ 
    appends brw weights series to df
    """
    count_returns = len(df["close"])-d # M
    weight_lst = [(1-decay_par)*(decay_par**j)/(1-decay_par**count_returns) for j in range(count_returns)]
    df["weight"] = pd.Series(weight_lst)
    return df

In [7]:
def get_ecdf(df):
    """
        assumes the input df have already passed through get_change_in_value_df and get_brw_weights_df
    """
    df = df.sort_values(by="change_in_value")
    df["ecdf"] = df["weight"].cumsum()
    
    return df

## Main Functions for DN, HS, BRW Approaches for a Single-Asset Portfolio

In [8]:
def d_day_alpha_percent_VaR_single_stock_delta_normal(df, d, N, alpha, ewma_par=None):
    
    """Returns the d-day 100(alpha)% VaR of a single stock using Delta Normal Approach.
    
    Parameters
    ----------
    df : pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row  
    d : int
        value of d for the d-day VaR
    N : int
        number of shares for the sole stock
    alpha : int
        the value to be used in calculting the 100(alpha)% VaR (e.g. 0.99, 0.95)
    ewma_par :float
        the value of the lambda parameter in an EWMA model for the volatility of the stock
        assumes the value is in the range (0,1)
    
    Returns
    -------
    float (2 decimal places)
        d-day 100(alpha)% VaR of a single stock using Delta Normal Approach
    """
    
    # gets the most recent stock price
    S_0 = df.loc[0, "close"]
    
    # append returns series to df
    df = get_return(df, d)
    
    #Case 1: if I will use EWMA
    if (ewma_par != None) and (0 < ewma_par < 1):
        df = get_ewma_weights_df(df, d, ewma_par)
        variance = np.nansum(df["weight"] * (df["return"]**2))
        sigma = np.sqrt(variance)
    #Case 2: I'll use the normal VaR
    else:
        # standard deviation of your stock returns
        sigma = df["return"].std()
    
    quantile = NormalDist().inv_cdf(alpha)
    var = N * S_0 * sigma * quantile
    
    return round(var, 2)

In [9]:
def d_day_alpha_percent_VaR_single_stock_historical(
    df, N, d, alpha
):
    """Returns the d-day 100(alpha)% VaR of a single stock using Historical Simulation Approach.
    
    Parameters
    ----------
    df : pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row  
    N : int
        number of shares for the sole stock
    d : int
        the value to be used in calculating the d-day VaR (e.g. 1-day, 5-day)
    alpha : float
        the value to be used in calculting the 100(alpha)% VaR (e.g. 0.99, 0.95)
    
    Returns
    -------
    float (2 decimal places)
        d-day 100(alpha)% VaR of a single stock using Historical Simulation Approach
    """
    
    # step 1: generate your historical returns
    df = get_return(df, d)
    # Step 2: get your historical changes in portfolio values
    df = get_change_in_value_df(df, N)
    # Step 3: get the (1-p/100)th percentile of the ordered array of historical changes in portfolio values
    var = get_kth_percentile_discrete(df, d, alpha)
    
    return round(var, 2)

In [10]:
def d_day_alpha_percent_VaR_single_stock_brw(
    df, N, d, alpha, decay_par
):
    """Returns the d-day 100*alpha% VaR of a single stock using BRW Approach.
    
    Parameters
    ----------
    df : pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row  
    N : int
        number of shares for the sole stock
    d : int
        the value to be used in calculating the d-day VaR (e.g. 1-day, 5-day)
    alpha : float
        the value to be used in calculting the 100(alpha)% VaR (e.g. 0.99, 0.95)
    decay_par : float
        the value of the BRW decay parameter
    
    Returns
    -------
    float (2 decimal places)
       d-day 100*alpha% VaR of a single stock using BRW Approach.
    """
    # STEP 1: ASSIGNMENT OF WEIGHTS
    # step 1.1: generate your historical returns
    df = get_return(df, d)
    # Step 1.2: get your historical changes in portfolio values
    df = get_change_in_value_df(df, N)
    # Step 1.3: generate the weights
    df = get_brw_weights_df(df, d, decay_par)
    
    # STEP 2: CONSTRUCT THE ECDF
    df = get_ecdf(df)
    
    # STEP 3: CALCULATE THE VAR
    var = abs(np.interp(1-alpha, df["ecdf"], df["change_in_value"]))
    
    return round(var, 2)

## Question:
Today is October 7, 2019. You have a long position on a portfolio of 20,000 US dollars. Assume that the one-day volatility of the stock follows an exponentially-weighted moving average model with parameter  $\lambda$=0.65. Also, suppose that the decay paramater for the BRW approach is 0.76. Determine the portfolio's one-day 99% VaR using:
1. DN with no EWMA
2. DN with EWMA
3. HS
4. BRW
 


In [11]:
USD_df = pd.read_csv("https://raw.githubusercontent.com/ateneomathdept/math100.2_2021Sem1/main/data/lectures/USDPHP.csv")
USD_df = USD_df.rename({'Mid':'close'}, axis=1)
USD_df

,Date,close
0,10/7/19,51.908
1,10/4/19,51.695
2,10/3/19,51.766
3,10/2/19,51.996
4,10/1/19,51.986
...,...,...
257,10/11/18,54.070
258,10/10/18,54.146
259,10/9/18,54.176
260,10/8/18,54.143


In [12]:
d_day_alpha_percent_VaR_single_stock_delta_normal(df=USD_df, d=1, N=20000, alpha=0.99, ewma_par=None)

8560.99

In [13]:
d_day_alpha_percent_VaR_single_stock_delta_normal(df=USD_df, d=1, N=20000, alpha=0.99, ewma_par=0.65)

8030.37

In [14]:
d_day_alpha_percent_VaR_single_stock_historical(df=USD_df, N=20000, d=1, alpha=0.99)

9211.84

In [15]:
 d_day_alpha_percent_VaR_single_stock_brw(df=USD_df, N=20000, d=1, alpha=0.99, decay_par=0.4)

4626.62